In [1]:
import tensorflow as tf

/Users/fatu/env/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
files = tf.data.Dataset.list_files(file_pattern)
dataset = tf.data.TFRecordDataset(files)

dataset = dataset.shuffle(10000)
dataset = dataset.repeat(NUM_EPOCHS)
dataset = dataset.map(lambda x: tf.parse_single_example(x, features))
dataset = dataset.batch(BATCH_SIZE)

or:

dataset = dataset.apply(
   tf.contrib.data.shuffle_and_repeat(10000, NUM_EPOCHS))
dataset = dataset.apply(
   tf.contrib.data.map_and_batch(lambda x: ..., BATCH_SIZE))

dataset = dataset.apply(tf.contrib.data.prefetch_to_device('/gpu:0'))
iterator = dataset.make_one_shot_iterator()
features = iterator.get_next()

* performance guideline
https://www.tensorflow.org/performance/datasets_performance

In [ ]:
tf.enable_eager_execution()

files = tf.data.Dataset.list_files(file_pattern)
dataset = tf.data.TFRecordDataset(files)
dataset = dataset.shuffle(10000)
dataset = dataset.repeat(NUM_EPOCHS)
dataset = dataset.map(lambda x : tf.parse_single_example(x, features))
dataset = dataset.batch(BATCH_SIZE)

In [ ]:
# Also implements best practivces for high performance!
# (See optional args for details.)
dataset = tf.contrib.data.make_batched_features_dataset(
   file_pattern, BATCH_SIZE, features, num_epochs=NUM_EPOCHS)

for batch in dataset:
   train_model(batch)

In [ ]:
# In a terminal, run the following commands, e.g. :
# $ pip install kaggle
# $ kaggle datasets download -d therohk/million-headlines -p .
dataset = tf.contrib.data.make_csv_dataset(
   "*.csv", BATCH_SIZE, num_epochs=NUM_EPOCHS)

for batch in dataset:
   train_model(batch["publish_date"], batch["headline_text"])

In [ ]:
# Wrap the dataset in an input function, and return it directly.
def input_fn():
   dataset = tf.contrib.data.make_csv_dataset(
      "*.csv", BATCH_SIZE, num_epochs=NUM_EPOCHS)
   return dataset

# Train an Estimator on the dataset.
tf.estimator.Estimator(model_fn=train_model).train(input_fn=input_fn)

For more information see:
https://www.tensorflow.org/performance/datasets_performance
https://www.tensorflow.org/programmers_guide/datasets